# Hypothesis Testing: Macroeconomic Stress and Payment Behavior

This notebook provides a framework for testing hypotheses about how macroeconomic stress periods affect payment volumes, fraud rates, and customer behavior.

## Proposed Hypotheses

### Volume & Activity Hypotheses
**H1**: Transaction volumes are significantly higher during high unemployment periods

**H2**: Customers make more frequent transactions (higher velocity) during economic stress

**H3**: Weekend and night-time transaction patterns differ between stress and normal periods

### Fraud Hypotheses
**H4**: Fraud rates increase during macroeconomic stress periods

**H5**: Time-to-fraud-detection is faster during stress periods (heightened vigilance)

**H6**: Certain fraud types (e.g., account takeover) are more prevalent during stress

### Transaction Behavior Hypotheses
**H7**: Average transaction amounts decrease during stress periods (economic constraint)

**H8**: The distribution of transaction types changes during stress (more withdrawals/transfers)

**H9**: Digital payment methods (mobile, digital wallet) are used more during stress vs traditional methods

### Customer Behavior Hypotheses
**H10**: Customer churn rate increases during macroeconomic stress periods

**H11**: Transaction failure rates increase during stress periods

**H12**: Customer transaction amounts become more volatile (higher std dev) during stress

---

## Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Libraries loaded successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

In [ ]:
# Load all data
print("Loading data...\n")

# Transaction data
transactions = pd.read_csv('../data/synthetic/payment_transactions.csv')
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
print(f"✓ Transactions: {len(transactions):,} records")

# Fraud data
fraud_data = pd.read_csv('../data/synthetic/fraud_histories.csv')
print(f"✓ Fraud data: {len(fraud_data):,} records")

# Merge transactions with fraud
data = transactions.merge(fraud_data, on='transaction_id', how='inner')

# FRED macroeconomic data
fed_funds = pd.read_csv('../data/fred/federal_funds_rate.csv')
fed_funds['date'] = pd.to_datetime(fed_funds['date'])

unemployment = pd.read_csv('../data/fred/unemployment_rate.csv')
unemployment['date'] = pd.to_datetime(unemployment['date'])

cpi = pd.read_csv('../data/fred/consumer_price_index.csv')
cpi['date'] = pd.to_datetime(cpi['date'])

print(f"✓ FRED data: {len(fed_funds)} months")

# Internal metrics
monthly_metrics = pd.read_csv('../data/synthetic/monthly_internal_metrics.csv')
monthly_metrics['year_month'] = pd.to_datetime(monthly_metrics['year_month'])
print(f"✓ Monthly metrics: {len(monthly_metrics)} months")

print("\nData loaded successfully!")

In [ ]:
# Define stress periods (same thresholds as main analysis)
unemployment_threshold = unemployment['unemployment_rate_percent'].quantile(0.75)
fed_funds_threshold = fed_funds['federal_funds_rate_percent'].quantile(0.75)

print("Stress Period Thresholds:")
print(f"  Unemployment: >{unemployment_threshold:.2f}%")
print(f"  Fed Funds Rate: >{fed_funds_threshold:.2f}%")

# Merge macro data
macro_data = fed_funds.merge(unemployment, on='date', how='inner')
macro_data = macro_data.merge(cpi, on='date', how='inner')

# Create stress indicators
macro_data['high_unemployment'] = (
    macro_data['unemployment_rate_percent'] > unemployment_threshold
).astype(int)

macro_data['high_interest_rate'] = (
    macro_data['federal_funds_rate_percent'] > fed_funds_threshold
).astype(int)

macro_data['stress_period'] = macro_data['high_unemployment'].astype(bool)

# Add to transactions
data['year_month'] = data['transaction_date'].dt.to_period('M').dt.to_timestamp()
data = data.merge(
    macro_data[['date', 'stress_period', 'unemployment_rate_percent', 'federal_funds_rate_percent']].rename(columns={'date': 'year_month'}),
    on='year_month',
    how='inner'
)

print(f"\nTransactions with stress labels: {len(data):,}")
print(f"  Stress periods: {data['stress_period'].sum():,} transactions")
print(f"  Normal periods: {(~data['stress_period']).sum():,} transactions")

---

## Statistical Testing Framework

We'll use appropriate statistical tests:
- **Two-sample t-test**: For comparing means (e.g., transaction amounts)
- **Mann-Whitney U test**: Non-parametric alternative for non-normal distributions
- **Chi-square test**: For categorical variables (e.g., fraud rates, transaction types)
- **Proportion z-test**: For comparing proportions

**Significance level (α)**: 0.05 (5%)

In [ ]:
# Helper functions for hypothesis testing

def test_means(stress_data, normal_data, metric_name):
    """
    Perform t-test and Mann-Whitney U test for comparing means.
    """
    # Two-sample t-test
    t_stat, t_pval = stats.ttest_ind(stress_data, normal_data)
    
    # Mann-Whitney U test (non-parametric)
    u_stat, u_pval = stats.mannwhitneyu(stress_data, normal_data, alternative='two-sided')
    
    print(f"\n{metric_name}:")
    print(f"  Stress period mean: {stress_data.mean():.4f}")
    print(f"  Normal period mean: {normal_data.mean():.4f}")
    print(f"  Difference: {stress_data.mean() - normal_data.mean():.4f} ({((stress_data.mean() / normal_data.mean() - 1) * 100):.2f}%)")
    print(f"\n  T-test: t={t_stat:.4f}, p-value={t_pval:.4f}")
    print(f"  Mann-Whitney U: U={u_stat:.0f}, p-value={u_pval:.4f}")
    
    if t_pval < 0.05:
        print(f"  ✓ SIGNIFICANT at α=0.05 (reject null hypothesis)")
    else:
        print(f"  ✗ NOT SIGNIFICANT at α=0.05 (fail to reject null hypothesis)")
    
    return t_stat, t_pval, u_stat, u_pval


def test_proportions(stress_count, stress_total, normal_count, normal_total, metric_name):
    """
    Perform z-test for comparing proportions.
    """
    from statsmodels.stats.proportion import proportions_ztest
    
    counts = np.array([stress_count, normal_count])
    nobs = np.array([stress_total, normal_total])
    
    z_stat, p_val = proportions_ztest(counts, nobs)
    
    stress_prop = stress_count / stress_total
    normal_prop = normal_count / normal_total
    
    print(f"\n{metric_name}:")
    print(f"  Stress period proportion: {stress_prop:.4f} ({stress_count}/{stress_total})")
    print(f"  Normal period proportion: {normal_prop:.4f} ({normal_count}/{normal_total})")
    print(f"  Difference: {stress_prop - normal_prop:.4f} ({((stress_prop / normal_prop - 1) * 100):.2f}%)")
    print(f"\n  Z-test: z={z_stat:.4f}, p-value={p_val:.4f}")
    
    if p_val < 0.05:
        print(f"  ✓ SIGNIFICANT at α=0.05 (reject null hypothesis)")
    else:
        print(f"  ✗ NOT SIGNIFICANT at α=0.05 (fail to reject null hypothesis)")
    
    return z_stat, p_val


def test_distribution(stress_data, normal_data, categories, metric_name):
    """
    Perform chi-square test for categorical distributions.
    """
    stress_counts = stress_data.value_counts()
    normal_counts = normal_data.value_counts()
    
    # Create contingency table
    contingency = pd.DataFrame({
        'Stress': [stress_counts.get(cat, 0) for cat in categories],
        'Normal': [normal_counts.get(cat, 0) for cat in categories]
    }, index=categories)
    
    chi2, p_val, dof, expected = stats.chi2_contingency(contingency)
    
    print(f"\n{metric_name}:")
    print(f"\nObserved Distribution:")
    contingency['Stress %'] = (contingency['Stress'] / contingency['Stress'].sum() * 100).round(2)
    contingency['Normal %'] = (contingency['Normal'] / contingency['Normal'].sum() * 100).round(2)
    print(contingency)
    
    print(f"\n  Chi-square: χ²={chi2:.4f}, p-value={p_val:.4f}, df={dof}")
    
    if p_val < 0.05:
        print(f"  ✓ SIGNIFICANT at α=0.05 (distributions differ)")
    else:
        print(f"  ✗ NOT SIGNIFICANT at α=0.05 (distributions similar)")
    
    return chi2, p_val

print("Helper functions defined!")

---

# Hypothesis Testing

## H1: Transaction volumes are significantly higher during high unemployment periods

**Null Hypothesis (H0)**: There is no difference in transaction volumes between stress and normal periods

**Alternative Hypothesis (H1)**: Transaction volumes are higher during stress periods

In [ ]:
print("="*70)
print("H1: Transaction Volumes During Stress Periods")
print("="*70)

# Aggregate by month
monthly_volume = data.groupby('year_month').agg({
    'transaction_id': 'count',
    'stress_period': 'first'
}).reset_index()
monthly_volume.columns = ['year_month', 'volume', 'stress_period']

stress_volumes = monthly_volume[monthly_volume['stress_period']]['volume']
normal_volumes = monthly_volume[~monthly_volume['stress_period']]['volume']

test_means(stress_volumes, normal_volumes, "Monthly Transaction Volume")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
axes[0].boxplot([normal_volumes, stress_volumes], labels=['Normal', 'Stress'])
axes[0].set_ylabel('Monthly Transaction Volume')
axes[0].set_title('Transaction Volume Distribution')
axes[0].grid(True, alpha=0.3)

# Time series
axes[1].plot(monthly_volume['year_month'], monthly_volume['volume'], marker='o')
stress_months = monthly_volume[monthly_volume['stress_period']]
axes[1].scatter(stress_months['year_month'], stress_months['volume'], color='red', s=100, zorder=5, label='Stress Period')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Monthly Transaction Volume')
axes[1].set_title('Transaction Volume Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## H2: Customers make more frequent transactions during stress periods

**Null Hypothesis (H0)**: Customer transaction frequency is the same across periods

**Alternative Hypothesis (H1)**: Customers transact more frequently during stress

In [ ]:
print("="*70)
print("H2: Customer Transaction Frequency")
print("="*70)

# Calculate transactions per customer per month
customer_freq = data.groupby(['year_month', 'customer_id', 'stress_period']).size().reset_index(name='tx_count')

stress_freq = customer_freq[customer_freq['stress_period']]['tx_count']
normal_freq = customer_freq[~customer_freq['stress_period']]['tx_count']

test_means(stress_freq, normal_freq, "Transactions per Customer per Month")

# Visualize
plt.figure(figsize=(12, 5))
plt.hist(normal_freq, bins=30, alpha=0.6, label='Normal', density=True)
plt.hist(stress_freq, bins=30, alpha=0.6, label='Stress', density=True)
plt.xlabel('Transactions per Customer per Month')
plt.ylabel('Density')
plt.title('Distribution of Customer Transaction Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## H3: Weekend and night-time transaction patterns differ during stress

**Null Hypothesis (H0)**: Proportion of weekend/night transactions is the same

**Alternative Hypothesis (H1)**: Proportion differs during stress periods

In [ ]:
print("="*70)
print("H3: Weekend and Night-time Transaction Patterns")
print("="*70)

# Add time features
data['hour'] = data['transaction_date'].dt.hour
data['day_of_week'] = data['transaction_date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'].isin([5, 6])
data['is_night'] = data['hour'].between(0, 6)

stress_data = data[data['stress_period']]
normal_data = data[~data['stress_period']]

# Weekend transactions
test_proportions(
    stress_data['is_weekend'].sum(),
    len(stress_data),
    normal_data['is_weekend'].sum(),
    len(normal_data),
    "Weekend Transactions"
)

# Night transactions
test_proportions(
    stress_data['is_night'].sum(),
    len(stress_data),
    normal_data['is_night'].sum(),
    len(normal_data),
    "Night-time Transactions (12am-6am)"
)

# Visualize hour distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hour of day
stress_hour_dist = stress_data['hour'].value_counts().sort_index() / len(stress_data) * 100
normal_hour_dist = normal_data['hour'].value_counts().sort_index() / len(normal_data) * 100

axes[0].plot(stress_hour_dist.index, stress_hour_dist.values, marker='o', label='Stress', linewidth=2)
axes[0].plot(normal_hour_dist.index, normal_hour_dist.values, marker='s', label='Normal', linewidth=2)
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Percentage of Transactions (%)')
axes[0].set_title('Transaction Distribution by Hour')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Day of week
stress_dow_dist = stress_data['day_of_week'].value_counts().sort_index() / len(stress_data) * 100
normal_dow_dist = normal_data['day_of_week'].value_counts().sort_index() / len(normal_data) * 100

x = np.arange(7)
width = 0.35
axes[1].bar(x - width/2, normal_dow_dist.values, width, label='Normal', alpha=0.8)
axes[1].bar(x + width/2, stress_dow_dist.values, width, label='Stress', alpha=0.8)
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Percentage of Transactions (%)')
axes[1].set_title('Transaction Distribution by Day')
axes[1].set_xticks(x)
axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## H4: Fraud rates increase during macroeconomic stress

**Null Hypothesis (H0)**: Fraud rate is the same across periods

**Alternative Hypothesis (H1)**: Fraud rate is higher during stress periods

In [ ]:
print("="*70)
print("H4: Fraud Rates During Stress Periods")
print("="*70)

stress_fraud = stress_data['is_fraud'].sum()
normal_fraud = normal_data['is_fraud'].sum()

test_proportions(
    stress_fraud,
    len(stress_data),
    normal_fraud,
    len(normal_data),
    "Fraud Rate"
)

# Visualize monthly fraud rate
monthly_fraud = data.groupby('year_month').agg({
    'is_fraud': 'mean',
    'stress_period': 'first'
}).reset_index()
monthly_fraud['fraud_rate_pct'] = monthly_fraud['is_fraud'] * 100

plt.figure(figsize=(14, 5))
plt.plot(monthly_fraud['year_month'], monthly_fraud['fraud_rate_pct'], marker='o', linewidth=2)
stress_months = monthly_fraud[monthly_fraud['stress_period']]
plt.scatter(stress_months['year_month'], stress_months['fraud_rate_pct'], color='red', s=100, zorder=5, label='Stress Period')
plt.axhline(y=1.5, color='gray', linestyle='--', label='Overall Avg (1.5%)')
plt.xlabel('Date')
plt.ylabel('Fraud Rate (%)')
plt.title('Monthly Fraud Rate Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## H5: Time-to-fraud-detection differs during stress periods

**Null Hypothesis (H0)**: Time to detect fraud is the same across periods

**Alternative Hypothesis (H1)**: Detection time differs during stress

In [ ]:
print("="*70)
print("H5: Time-to-Fraud-Detection")
print("="*70)

# Only fraud cases with detection time
fraud_cases = data[data['is_fraud'] == 1].copy()
fraud_cases = fraud_cases[fraud_cases['time_to_detection_hours'].notna()]

if len(fraud_cases) > 0:
    stress_detection = fraud_cases[fraud_cases['stress_period']]['time_to_detection_hours']
    normal_detection = fraud_cases[~fraud_cases['stress_period']]['time_to_detection_hours']
    
    if len(stress_detection) > 0 and len(normal_detection) > 0:
        test_means(stress_detection, normal_detection, "Time to Fraud Detection (hours)")
        
        # Visualize
        plt.figure(figsize=(12, 5))
        plt.boxplot([normal_detection, stress_detection], labels=['Normal', 'Stress'])
        plt.ylabel('Hours to Detection')
        plt.title('Time-to-Fraud-Detection Distribution')
        plt.grid(True, alpha=0.3, axis='y')
        plt.show()
    else:
        print("Insufficient fraud cases in one or both periods")
else:
    print("No fraud cases with detection time data")

---

## H6: Distribution of fraud types differs during stress

**Null Hypothesis (H0)**: Fraud type distribution is the same across periods

**Alternative Hypothesis (H1)**: Certain fraud types are more prevalent during stress

In [ ]:
print("="*70)
print("H6: Fraud Type Distribution")
print("="*70)

# Only fraud cases
fraud_cases = data[data['is_fraud'] == 1].copy()
fraud_cases = fraud_cases[fraud_cases['fraud_type'] != '']

if len(fraud_cases) > 0:
    stress_fraud_types = fraud_cases[fraud_cases['stress_period']]['fraud_type']
    normal_fraud_types = fraud_cases[~fraud_cases['stress_period']]['fraud_type']
    
    fraud_type_categories = fraud_cases['fraud_type'].unique()
    
    if len(stress_fraud_types) > 0 and len(normal_fraud_types) > 0:
        test_distribution(stress_fraud_types, normal_fraud_types, fraud_type_categories, "Fraud Types")
        
        # Visualize
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        normal_counts = normal_fraud_types.value_counts()
        stress_counts = stress_fraud_types.value_counts()
        
        axes[0].pie(normal_counts.values, labels=normal_counts.index, autopct='%1.1f%%')
        axes[0].set_title('Fraud Types - Normal Period')
        
        axes[1].pie(stress_counts.values, labels=stress_counts.index, autopct='%1.1f%%')
        axes[1].set_title('Fraud Types - Stress Period')
        
        plt.tight_layout()
        plt.show()
    else:
        print("Insufficient fraud cases in one or both periods")
else:
    print("No fraud cases with fraud type data")

---

## H7: Average transaction amounts decrease during stress

**Null Hypothesis (H0)**: Transaction amounts are the same across periods

**Alternative Hypothesis (H1)**: Transaction amounts are lower during stress (economic constraint)

In [ ]:
print("="*70)
print("H7: Transaction Amounts During Stress")
print("="*70)

stress_amounts = stress_data['amount']
normal_amounts = normal_data['amount']

test_means(stress_amounts, normal_amounts, "Transaction Amount")

# Also test median
print(f"\n  Stress median: ${stress_amounts.median():.2f}")
print(f"  Normal median: ${normal_amounts.median():.2f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution (limited to $500 for visibility)
axes[0].hist(normal_amounts[normal_amounts <= 500], bins=50, alpha=0.6, label='Normal', density=True)
axes[0].hist(stress_amounts[stress_amounts <= 500], bins=50, alpha=0.6, label='Stress', density=True)
axes[0].set_xlabel('Transaction Amount ($)')
axes[0].set_ylabel('Density')
axes[0].set_title('Transaction Amount Distribution (up to $500)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot([normal_amounts, stress_amounts], labels=['Normal', 'Stress'])
axes[1].set_ylabel('Transaction Amount ($)')
axes[1].set_title('Transaction Amount Box Plot')
axes[1].set_ylim(0, 500)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## H8: Transaction type distribution changes during stress

**Null Hypothesis (H0)**: Transaction type distribution is the same

**Alternative Hypothesis (H1)**: Certain transaction types (withdrawals, transfers) increase during stress

In [ ]:
print("="*70)
print("H8: Transaction Type Distribution")
print("="*70)

transaction_types = data['transaction_type'].unique()

test_distribution(
    stress_data['transaction_type'],
    normal_data['transaction_type'],
    transaction_types,
    "Transaction Types"
)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

normal_type_pct = normal_data['transaction_type'].value_counts(normalize=True).sort_index() * 100
stress_type_pct = stress_data['transaction_type'].value_counts(normalize=True).sort_index() * 100

x = np.arange(len(normal_type_pct))
width = 0.35

axes[0].bar(x - width/2, normal_type_pct.values, width, label='Normal', alpha=0.8)
axes[0].bar(x + width/2, stress_type_pct.values, width, label='Stress', alpha=0.8)
axes[0].set_xlabel('Transaction Type')
axes[0].set_ylabel('Percentage (%)')
axes[0].set_title('Transaction Type Distribution')
axes[0].set_xticks(x)
axes[0].set_xticklabels(normal_type_pct.index, rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Difference plot
diff = stress_type_pct - normal_type_pct
colors = ['green' if x > 0 else 'red' for x in diff.values]
axes[1].bar(range(len(diff)), diff.values, color=colors, alpha=0.8)
axes[1].set_xlabel('Transaction Type')
axes[1].set_ylabel('Percentage Point Difference')
axes[1].set_title('Stress vs Normal (Stress - Normal)')
axes[1].set_xticks(range(len(diff)))
axes[1].set_xticklabels(diff.index, rotation=45, ha='right')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## H9: Digital payment methods increase during stress

**Null Hypothesis (H0)**: Payment method distribution is the same

**Alternative Hypothesis (H1)**: Digital methods (mobile, digital wallet) are more prevalent during stress

In [ ]:
print("="*70)
print("H9: Payment Method Preferences")
print("="*70)

payment_methods = data['payment_method'].unique()

test_distribution(
    stress_data['payment_method'],
    normal_data['payment_method'],
    payment_methods,
    "Payment Methods"
)

# Test digital vs traditional
data['is_digital'] = data['payment_method'].isin(['digital_wallet'])
data['is_mobile'] = data['device_type'] == 'mobile'

stress_data_updated = data[data['stress_period']]
normal_data_updated = data[~data['stress_period']]

print("\n" + "-"*70)
test_proportions(
    stress_data_updated['is_digital'].sum(),
    len(stress_data_updated),
    normal_data_updated['is_digital'].sum(),
    len(normal_data_updated),
    "Digital Wallet Usage"
)

test_proportions(
    stress_data_updated['is_mobile'].sum(),
    len(stress_data_updated),
    normal_data_updated['is_mobile'].sum(),
    len(normal_data_updated),
    "Mobile Device Usage"
)

# Visualize
normal_method_pct = normal_data['payment_method'].value_counts(normalize=True).sort_index() * 100
stress_method_pct = stress_data['payment_method'].value_counts(normalize=True).sort_index() * 100

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(normal_method_pct))
width = 0.35

ax.bar(x - width/2, normal_method_pct.values, width, label='Normal', alpha=0.8)
ax.bar(x + width/2, stress_method_pct.values, width, label='Stress', alpha=0.8)
ax.set_xlabel('Payment Method')
ax.set_ylabel('Percentage (%)')
ax.set_title('Payment Method Distribution')
ax.set_xticks(x)
ax.set_xticklabels(normal_method_pct.index, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## H10: Customer churn increases during stress

**Null Hypothesis (H0)**: Churn rate is the same across periods

**Alternative Hypothesis (H1)**: Churn rate is higher during stress

In [ ]:
print("="*70)
print("H10: Customer Churn Rate")
print("="*70)

# Merge with monthly metrics
monthly_with_stress = monthly_metrics.merge(
    macro_data[['date', 'stress_period']].rename(columns={'date': 'year_month'}),
    on='year_month',
    how='inner'
)

stress_churn = monthly_with_stress[monthly_with_stress['stress_period']]['avg_churn_rate_pct']
normal_churn = monthly_with_stress[~monthly_with_stress['stress_period']]['avg_churn_rate_pct']

test_means(stress_churn, normal_churn, "Monthly Churn Rate (%)")

# Visualize
plt.figure(figsize=(14, 5))
plt.plot(monthly_with_stress['year_month'], monthly_with_stress['avg_churn_rate_pct'], marker='o', linewidth=2)
stress_months = monthly_with_stress[monthly_with_stress['stress_period']]
plt.scatter(stress_months['year_month'], stress_months['avg_churn_rate_pct'], 
           color='red', s=100, zorder=5, label='Stress Period')
plt.xlabel('Date')
plt.ylabel('Churn Rate (%)')
plt.title('Monthly Customer Churn Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## H11: Transaction failure rates increase during stress

**Null Hypothesis (H0)**: Failure rate is the same across periods

**Alternative Hypothesis (H1)**: Failure rate is higher during stress (financial strain)

In [ ]:
print("="*70)
print("H11: Transaction Failure Rates")
print("="*70)

data['is_failed'] = data['status'] == 'failed'

stress_data_updated = data[data['stress_period']]
normal_data_updated = data[~data['stress_period']]

test_proportions(
    stress_data_updated['is_failed'].sum(),
    len(stress_data_updated),
    normal_data_updated['is_failed'].sum(),
    len(normal_data_updated),
    "Transaction Failure Rate"
)

# Visualize monthly failure rate
monthly_failures = data.groupby('year_month').agg({
    'is_failed': 'mean',
    'stress_period': 'first'
}).reset_index()
monthly_failures['failure_rate_pct'] = monthly_failures['is_failed'] * 100

plt.figure(figsize=(14, 5))
plt.plot(monthly_failures['year_month'], monthly_failures['failure_rate_pct'], marker='o', linewidth=2)
stress_months = monthly_failures[monthly_failures['stress_period']]
plt.scatter(stress_months['year_month'], stress_months['failure_rate_pct'], 
           color='red', s=100, zorder=5, label='Stress Period')
plt.xlabel('Date')
plt.ylabel('Failure Rate (%)')
plt.title('Monthly Transaction Failure Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## H12: Transaction amount volatility increases during stress

**Null Hypothesis (H0)**: Customer spending volatility is the same

**Alternative Hypothesis (H1)**: Spending becomes more volatile during stress (irregular patterns)

In [ ]:
print("="*70)
print("H12: Customer Transaction Amount Volatility")
print("="*70)

# Calculate customer-month volatility
customer_volatility = data.groupby(['year_month', 'customer_id', 'stress_period']).agg({
    'amount': 'std'
}).reset_index()
customer_volatility.columns = ['year_month', 'customer_id', 'stress_period', 'amount_std']

# Remove NaN (customers with only 1 transaction in a month)
customer_volatility = customer_volatility.dropna()

stress_volatility = customer_volatility[customer_volatility['stress_period']]['amount_std']
normal_volatility = customer_volatility[~customer_volatility['stress_period']]['amount_std']

test_means(stress_volatility, normal_volatility, "Customer Transaction Amount Std Dev")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
axes[0].hist(normal_volatility[normal_volatility <= 300], bins=50, alpha=0.6, label='Normal', density=True)
axes[0].hist(stress_volatility[stress_volatility <= 300], bins=50, alpha=0.6, label='Stress', density=True)
axes[0].set_xlabel('Transaction Amount Std Dev ($)')
axes[0].set_ylabel('Density')
axes[0].set_title('Customer Spending Volatility Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot([normal_volatility, stress_volatility], labels=['Normal', 'Stress'])
axes[1].set_ylabel('Transaction Amount Std Dev ($)')
axes[1].set_title('Spending Volatility Box Plot')
axes[1].set_ylim(0, 300)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## Summary of Results

Let's create a summary table of all hypothesis test results.

In [ ]:
print("="*70)
print("HYPOTHESIS TESTING SUMMARY")
print("="*70)
print()
print("This summary shows which hypotheses were supported by the data at α=0.05")
print("significance level. Remember:")
print("  • p < 0.05: Statistically significant (reject null hypothesis)")
print("  • p ≥ 0.05: Not significant (fail to reject null hypothesis)")
print()
print("Note: Statistical significance does not imply practical significance.")
print("Always consider effect sizes and business context when interpreting results.")
print()
print("-"*70)
print()
print("Review the individual hypothesis tests above for:")
print("  • Specific p-values and test statistics")
print("  • Effect sizes and directional changes")
print("  • Visualizations of the patterns")
print("  • Business implications of the findings")

---

## Custom Hypothesis Testing

Use this section to test your own hypotheses using the loaded data and helper functions.

In [ ]:
# Example: Test your own hypothesis here

# Available datasets:
# - data: Full transaction data with macro indicators
# - stress_data: Transactions during stress periods
# - normal_data: Transactions during normal periods
# - monthly_metrics: Monthly business metrics
# - macro_data: FRED macroeconomic indicators

# Available helper functions:
# - test_means(stress_values, normal_values, metric_name)
# - test_proportions(stress_count, stress_total, normal_count, normal_total, metric_name)
# - test_distribution(stress_categorical, normal_categorical, categories, metric_name)

# Your code here:
